In [ ]:
# Seed value
# Apparently you may use different seed values at each stage
seed_value= 0

# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
import os
%env PYTHONHASHSEED=0

# 2. Set `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.random.set_seed(seed_value)
# for later versions: 
# tf.compat.v1.set_random_seed(seed_value)

# 5. Configure a new global `tensorflow` session

# for later versions:
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

In [ ]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras import losses
from tensorflow.keras import optimizers
from tensorflow.keras import metrics

from tensorflow.keras.layers import Input, Conv1D, Conv2D, Flatten, Dense, Conv1DTranspose, Conv2DTranspose, Reshape, Lambda, Activation, BatchNormalization, LeakyReLU, Dropout, ReLU, Masking, Concatenate, Layer
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import Zeros
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import plot_model

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.python.keras import regularizers

from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.inspection import permutation_importance

import lime.lime_tabular

import re

from sklearn.model_selection import train_test_split

import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)

In [ ]:
meta = pd.read_csv("cdiff_meta_COMBINED.txt", sep = "\t")

#bacterial datasets
bacteria = pd.read_csv("filtered_species_table-NEW.txt", sep = "\t")
bacteria_species = pd.read_csv("ALL-SPECIES-species-ed_lefse_CDIStatus-k2-full.txt.2.txt", sep = "\t")

#fungal datasets
fungi = pd.read_csv("filtered_species_table-FUNGI.txt", sep = "\t")
fungi_species = pd.read_csv("FUNGI-SPECIES-species-lefse_CDIStatus-k2-fungi.txt.2.txt", sep = "\t")
fungi_genes = pd.read_csv("FUNGI-GENES-ed_renamed_lefse_CDIStatus-genes-emapper-fungi.txt.2.txt", sep = "\t")
full_fungi = pd.read_csv("taxa-expression-profile-FUNGI.txt", sep = "\t")

#combined datasets
combined = pd.read_csv("table-hits-combined.txt", sep = "\t")
combined_genes = pd.read_csv("ALL-GENES-renamed_lefse_CDIStatus-emapper-full.txt", sep = "\t")

In [ ]:
fungi_species_name = []
for species in fungi_species["Species"]:
    species = species.split('.')
    species = ';'.join(species)
    fungi_species_name.append(species)

In [ ]:
# transpose fungi data
fungi = fungi.set_index("SampleID")
fungi = fungi.transpose()
fungi.index = fungi.index.map(int)

In [ ]:
fungi = fungi[fungi_species_name]

In [ ]:
# sort by index
fungi = fungi.sort_index()

In [ ]:
# transpose fungi data
full_fungi = full_fungi.set_index("SampleID")
full_fungi = full_fungi.transpose()
full_fungi.index = full_fungi.index.map(int)

In [ ]:
full_fungi = full_fungi[fungi_genes["GeneID"]]
full_fungi.shape

In [ ]:
full_fungi = full_fungi.sort_index()
full_fungi_names = full_fungi.columns

In [ ]:
bacteria_species_names = []
for name in bacteria_species["Species"]:
    name = re.sub("[^0-9a-zA-Z]+", "_", name)
    bacteria_species_names.append(name)

In [ ]:
# transpose bacteria data
bacteria = bacteria.set_index("SampleID")
bacteria = bacteria.transpose()

In [ ]:
# get sample indexes
sample_ids = list(bacteria.index)

In [ ]:
# sort by index
bacteria.index = bacteria.index.map(int)
bacteria = bacteria.sort_index()

In [ ]:
new_col_names = []
for name in bacteria.columns:
    name = re.sub("[^0-9a-zA-Z]+", "_", name)
    new_col_names.append(name)
    
bacteria.columns = new_col_names

In [ ]:
bacteria = bacteria[bacteria_species_names]

In [ ]:
bacteria.shape

In [ ]:
# transpose combined data
combined = combined.set_index("SampleID")
combined = combined.transpose()
combined.index = combined.index.map(int)

In [ ]:
combined = combined[combined_genes["GeneID"]]
combined.shape

In [ ]:
combined = combined.sort_index()
combined_names = combined.columns

In [ ]:
meta = meta[meta["SampleID"].isin(sample_ids)]

In [ ]:
# make output vector (positive/negative) from metadata
y = [1 if i == 'Positive' else 0 for i in meta["CDIStatus"]]
y = np.array(y)
y = y.astype("float32")

In [ ]:
#convert to numpy array
fungi = np.array(fungi)
fungi = fungi.astype("float32")

full_fungi = np.array(full_fungi)
full_fungi = full_fungi.astype("float32")

combined = np.array(combined)
combined = combined.astype("float32")

bacteria = np.array(bacteria)
bacteria = bacteria.astype("float32")

In [ ]:
combined_species = np.array(np.hstack([fungi, bacteria]))
combined_species.shape

In [ ]:
combined_species_names = fungi_species_name + bacteria_species_names

In [ ]:
# Linear SVM
from sklearn import svm
from sklearn import metrics
import matplotlib.pyplot as plt

clf = svm.SVC(kernel = 'linear', C = 1)
clf.fit(combined_species, y)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=0)
score = cross_val_score(clf, combined_species, y, scoring='accuracy', cv=cv)

# report performance
print(f'Scores for each fold are: {score}')
print(f'Average score: {"{:.3f} (+/-{:.3f})".format(score.mean(), score.std())}')

pd.Series(abs(clf.coef_[0]), index=combined_species_names).nlargest(10).plot(kind='barh')


In [ ]:
# Random Forest 
clf = RandomForestClassifier()
clf.fit(combined_species, y)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=1)
score = cross_val_score(clf, combined_species, y, scoring='accuracy', cv=cv)

# report performance
print(f'Scores for each fold are: {score}')
print(f'Average score: {"{:.3f} (+/-{:.3f})".format(score.mean(), score.std())}')

pd.Series(abs(clf.feature_importances_), index=combined_species_names).nlargest(10).plot(kind='barh')

In [ ]:
# Now with cross-validation
# Begin K-fold Cross-Validation
kfold = RepeatedStratifiedKFold(n_splits = 10, n_repeats = 5, random_state = 0)
cvacc = []
cvauc = []

# create model with CV compatibility
for train, test in kfold.split(combined_species, y):
    
    model = Sequential([
        Lambda(lambda x: K.log(K.cast((1 + x * 1000), "float32"))/K.log(K.cast((1 + 1000), "float32"))),
        Dense(500, activation = 'softplus', kernel_regularizer=tf.keras.regularizers.l1(.001)),
        Dropout(.4),
        BatchNormalization(),
        Dense(500, activation = 'softplus', kernel_regularizer=tf.keras.regularizers.l1(.001)),
        Dropout(.4),
        BatchNormalization(),
        Dense(500, activation = 'softplus', kernel_regularizer=tf.keras.regularizers.l1(.001)),
        Dropout(.4),
        BatchNormalization(),
        Dense(500, activation = 'softplus', kernel_regularizer=tf.keras.regularizers.l1(.001)),
        Dropout(.4),
        BatchNormalization(),
        Dense(500, activation = 'softplus', kernel_regularizer=tf.keras.regularizers.l1(.001)),
        Dropout(.4),
        BatchNormalization(),
        Dense(500, activation = 'softplus', kernel_regularizer=tf.keras.regularizers.l1(.001)),
        Dropout(.4),
        BatchNormalization(),
        Dense(1, activation = 'sigmoid')
    ])
    
    model.compile(optimizer = "Adam",
                  loss = "binary_crossentropy",
                  metrics = ["accuracy"])
    
    model.fit(combined_species[train], y[train],
              epochs = 1000,
              validation_split = .2,
              verbose = 0)
    
    train_result = model.evaluate(combined_species[train], y[train], verbose = 0)
    print("train {:s}: {:.2%} ".format(model.metrics_names[1], train_result[1]))
    
    result = model.evaluate(combined_species[test], y[test], verbose = 0)
    print("{:s}: {:.2%} \n".format(model.metrics_names[1], result[1]))
    
    cvacc.append(result[1])
    
print("Accuracy: {:.2%} (+/-{:.2%})".format(np.mean(cvacc), np.std(cvacc)))